<a href="https://colab.research.google.com/github/DrAlexSanz/NLP-SPEC-C2/blob/master/W2/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download everything:
!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W2/WSJ_02-21.pos
!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W2/WSJ_24.pos
!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W2/hmm_vocab.txt
!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W2/test.words.txt
!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W2/utils_pos.py

--2020-10-07 20:52:08--  https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W2/WSJ_02-21.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8279089 (7.9M) [text/plain]
Saving to: ‘WSJ_02-21.pos’

WSJ_02-21.pos       100%[===================>]   7.90M  17.4MB/s    in 0.5s    

2020-10-07 20:52:09 (17.4 MB/s) - ‘WSJ_02-21.pos’ saved [8279089/8279089]

--2020-10-07 20:52:09--  https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W2/WSJ_24.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286063 (279K) [text/plain]
Sa

In [3]:
import math
import pandas as pd
import numpy as np
from utils_pos import get_word_tag, preprocess
from collections import defaultdict

print("Everything imported correctly")

Everything imported correctly
